In [ ]:
!pip install -q -U transformers datasets bitsandbytes peft trl huggingface_hub

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import torch
from datasets import DatasetDict

MODEL_NAME = "Cagatayd/llama3.2-1B-Instruct-Egitim"

## Step 1: Prepare Dataset

In [ ]:
dataset = load_dataset("MattCoddity/dockerNLcommands")

train_val_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

dataset = DatasetDict(
    {'train': train_val_split['train'],
     'validation': train_val_split['test']}
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="left",trust_remote_code= True)

def tokenize_fn(example: dict) -> dict:
  return tokenizer(tokenizer.apply_chat_template(example['text'], tokenize=False))

def format(example: dict) -> dict:
    messages = [
        {"role": 'system','content':example['instruction']},
        {"role": 'user','content':example['input']},
        {"role": 'assistant','content':example['output']}

    ]

    return {'text':messages}

In [ ]:
dataset = dataset.map(format)
tokenized_dataset = dataset.map(tokenize_fn,batched=True,remove_columns=['input', 'output', 'instruction', 'text'])

## Step 2: Load Quantized Model

In [ ]:
config_8bit = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=config_8bit,
    device_map= "auto",
    trust_remote_code =True
)

model_8bit

## Step 3: Set-up LORA Config

In [ ]:
from peft import  LoraConfig, get_peft_model

lora_config = LoraConfig(
    r= 64,
    lora_alpha = 32,
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"

)

model_8bit_lora = get_peft_model(model_8bit,lora_config)
model_8bit_lora.print_trainable_parameters()

## Step 4: Set up SFT Trainer

In [7]:
from transformers import TrainingArguments
from trl import SFTTrainer


training_args = TrainingArguments(
    output_dir = "./resutls",
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    eval_steps = 10,
    eval_strategy = "steps",
    save_steps = 20,
    save_strategy = "steps",
    # num_train_epochs = 1,
    max_steps = 60,
    learning_rate = 3e-5,
    weight_decay = 0.01,
    warmup_steps = 5,
    fp16 = True,
    gradient_accumulation_steps = 4,
    optim = "adamw_torch",
    logging_steps = 10,
    report_to = "none"
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False,return_tensors="pt")

tokenizer.pad_token = tokenizer.eos_token

trainer = SFTTrainer(
    model =model_8bit_lora,
    train_dataset =tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    args = training_args,
    data_collator= data_collator,
    processing_class = tokenizer
)

## Step 5: Train & Save

In [ ]:
trainer.train()

In [11]:
model_8bit_lora.save_pretrained("model_8bit_sft")